In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
import math
import numpy as np
import numpy.matlib
import random
import os
import pathlib
import time

## Classes

In [17]:
class Individual:
  def __init__(self,c, w, fitness=math.inf):
    self.c = c
    self.w = w
    self.fitness = fitness

  # Getters
  def getFitness(self):
    return self.fitness

  # Setters
  def setFitness(self,fitness):
    self.fitness = fitness
  
  # Methods
  def printGenome(self):
    return print("C (size): {0}, W (size): {1}\nC: {2}\nW: {3}\n fitness: {4}".format(self.c.shape, self.w.shape, self.c, self.w, self.fitness))

In [18]:
class Population:
  def __init__(self,size):
    self.size = size
    self.geneList = []
    self.fitnessList = []
    self.bestGene = None

  # Methods
  def addGeneToList(self,gen):
    self.geneList.append(gen);

  def addFitnessToList(self,fitness):
    self.fitnessList.append(fitness);

  def insertBestGene(self,gen):
    self.bestGene = gen
    self.geneList.append(gen)
    self.fitnessList.append(gen.fitness)

  def initializePopulation(self):
    for i in range(self.size):
      gen = Individual(random_C_generator(K), random_W_generator(K,optimalD))
      self.geneList.append(gen)
    
    # Once the population is set, we choose a random best gene
    randomIndex = np.random.randint(low=0, high=self.size)
    self.bestGene = self.geneList[randomIndex]

    return self

In [19]:
class Particle(Individual):
  def __init__(self,c, w, fitness=math.inf):
    super().__init__(c, w, fitness)
    self.bestFitness = fitness
    self.wPosition = w
    self.cPosition = c
    self.wVelocity = 0 # All swarm's velocities are set to 0 at the start
    self.cVelocity = 0 

  # Setters
  def setNewWeightsPosition(self, new_W):
    self.wPosition = new_W
    
  def setNewChromosomePosition(self, new_C):
    self.cPosition = new_C

  def setNewWeightsVelocity(self, new_W):
    self.wVelocity = new_W

  def setNewChromosomeVelocity(self, new_C):
    self.cVelocity = new_C

  def setBestFitness(self,newF):
    self.bestFitness = newF

In [20]:
class Swarm(Population):
  def __init__(self,size):
    super().__init__(size)
    self.globalBestW = None
    self.globalBestC = None

  # Getters
  def getBestSwarmWeights(self):
    return self.globalBestW

  def getBestSwarmChromosome(self):
    return self.globalBestC

  # Setters
  def setBestSwarmWeigths(self,newW):
    self.globalBestW = newW

  def setBestSwarmChromosome(self,newC):
    self.globalBestC = newC

  def initializeSwarm(self):
    for i in range(self.size):
      p = Particle(random_C_generator(K), random_W_generator(K,optimalD))
      self.geneList.append(p)
          
      # Once the swarm is set, we choose a random best gene
    randomIndex = np.random.randint(low=0, high=self.size)
    self.bestGene = self.geneList[randomIndex]
    self.globalBestC = self.bestGene.c
    self.globalBestW = self.bestGene.w

    return self

In [21]:
class Reef:
  def __init__(self,rho0, R):
    self.reefSize = R
    self.freeOccupiedRate = rho0
    self.geneList = np.full([R,1], None)
    self.fitnessList = []
    self.bestGene = None

  # Methods
  def addGeneToList(self,gen):
    self.geneList.append(gen);

  def addFitnessToList(self,fitness):
    self.fitnessList.append(fitness);

  def insertBestGene(self,gen):
    self.bestGene = gen
    self.geneList.append(gen)
    self.fitnessList.append(gen.fitness)

  def initializeReef(self):
    occupiedHoles = int(self.reefSize * self.freeOccupiedRate)# Partially Occupied
    print(occupiedHoles)
    for i in range(occupiedHoles): 
      # Choose random hole
      random_hole_index = np.random.randint(0,self.geneList.shape[0])
      
      while 1: # If that hole is occupied then choose again until it finds an empty one 
        if not self.geneList[random_hole_index][0]:
          break
        else:
          random_hole_index = np.random.randint(0,self.geneList.shape[0])

      # Insert Individual in that hole
      larvae = Individual(random_C_generator(K), random_W_generator(K,optimalD))
      self.geneList[random_hole_index][0] = larvae
      
      # Once the population is set, we choose a random best gene
      random_best_hole_index = np.random.randint(0,self.geneList.shape[0])
      
      while 1: # If that hole is occupied then choose again until it finds an empty one 
        if not self.geneList[random_best_hole_index][0]:
          self.bestGene = self.geneList[random_best_hole_index][0]
          break
        else:
          random_best_hole_index = np.random.randint(0,self.geneList.shape[0])

    return self

## Functions

### NumPy custom functions

In [22]:
def sumMatrixes(mat1,mat2):
  return np.add(mat1,np.asarray(mat2))

def subtractMatrixes(mat1,mat2):
  return np.subtract(mat1,np.asarray(mat2))

def dotMultiply(mat1,mat2):
  return np.multiply(mat1,np.asarray(mat2))

def multiplyMatrixes(mat1,mat2):
  return np.matmul(mat1,np.asarray(mat2))

def dotDivide(mat1,mat2):
  return np.divide(mat1,np.asarray(mat2))

### Population

In [23]:
def random_C_generator(K):
  # numpy.random.randint/uniform returns from: [low,high).
  return np.random.randint(low= 0, high= 1 + 1, size=[1,K])

def random_W_generator(K,D):
  return np.random.uniform(low= -1, high= 1, size=[K,D])

#### Reproduction

In [24]:
def InternalReproduction(gen):

  # Mutate chromosome 
  mutate_point = np.random.randint(0, len(gen.c)) 
  
  if(gen.c[0][mutate_point] == 0):
    gen.c[0][mutate_point] = 1
  elif(gen.c[0][mutate_point] == 1):
    gen.c[0][mutate_point] = 0

  # Mutate weights
  nRows = gen.w.shape[0]
  nColumns = gen.w.shape[1]
  mutate_column = np.random.randint(0, nColumns) 
  
  gen.w[:,mutate_column] = np.random.uniform(-1,1,nRows)

In [25]:
def ExternalReproduction(father,mother):
  crossover_c_point = np.random.randint(0, len(father.c))
  crossover_column_w_point = random.randint(0, len(father.w[0]))

  # Offsprings
  offspring1 = Individual( c = np.concatenate((father.c[:crossover_c_point], mother.c[crossover_c_point:])), 
                           w = np.concatenate((father.w[:,:crossover_column_w_point], mother.w[:,crossover_column_w_point:]), axis=1) )
  
  offspring2 = Individual( c = np.concatenate((mother.c[:crossover_c_point], father.c[crossover_c_point:])), 
                           w = np.concatenate((mother.w[:,:crossover_column_w_point], father.w[:,crossover_column_w_point:]), axis=1) )
                       
  return [offspring1,offspring2]

In [26]:
def RouletteWheelSelection(population): 
  # Create a population for the chosen individuals of the Roulette
  roulettePopulation = Population(population.size)

  # Make sure we don't lose our best gene in the roulette 
  roulettePopulation.insertBestGene(population.bestGene)

  # Total population fitness (S)
  S = sum([chromosome.fitness for chromosome in population.geneList])
  
  # Population chromosomes' relative probabilities
  rel_prob = [chromosome.fitness/S for chromosome in population.geneList]

  for i in range(population.size - 1):
    # Generate a random uniform number (r) - (0,1]
    r = np.random.uniform()
    # Find the first index for which q_i < r
    for index,individual in enumerate(population.geneList):
      r -= rel_prob[index]
      if r < 0:
        roulettePopulation.addGeneToList(individual)
        roulettePopulation.addFitnessToList(individual.fitness)
        break

  return roulettePopulation 

In [27]:
# This functions reproduces a population.
# It crossover the parents (external reproduction) and mutate the offspring (internal reproduction).
# Returns the input population, updated with the crossover and mutated childs

# ACK Works Great
def ReproducePopulation(population):

  childs = [] # list to store future offsprings
  crossoverProb = np.random.uniform() # crossover probability
  mutationProb = np.random.uniform() # mutation probability
  geneListSize = len(population.geneList)

  for index,individual in enumerate(population.geneList):
    # Generate a random number and check if it's below the crossover probability
    if(random.random() < crossoverProb):
      random_mother_index = np.random.randint(0,geneListSize)

      while(random_mother_index == index): # Make sure that the mother is a different individual in the population
        random_mother_index = np.random.randint(0,geneListSize)

      mother = population.geneList[random_mother_index]

      # Crossover parents
      offsprings = ExternalReproduction(individual, mother)

      # Mutate offsprings
      if(random.random() < mutationProb): 
        InternalReproduction(offsprings[0])

      if(random.random() < mutationProb):
        InternalReproduction(offsprings[1])

      childs.append(offsprings[0])
      childs.append(offsprings[1])
  
  # Update population's Genes list with newly-created childs
  population.geneList = np.concatenate((population.geneList, childs))

  return population

#### Swarm

In [28]:
def UpdateVelocitiesAndPositionsPSO(swarm, w, c1, c2):
  for p in swarm.geneList:
    # Velocities
    r1 = np.random.uniform()
    r2 = np.random.uniform()

    cVt_i = (w * p.cVelocity) + (c1*r1*(p.bestFitness - p.cPosition)) + (c2*r2*(swarm.bestGene.cPosition - p.cPosition))
    wVt_i = (w * p.wVelocity) + (c1*r1*(p.bestFitness - p.wPosition)) + (c2*r2*(swarm.bestGene.wPosition - p.wPosition))

    print("cVt_i:",cVt_i)
    print("wVt_i:",wVt_i)

    # Positions
    velocityProbability = 2 / math.pi * math.atan((math.pi*0.5)*cVt_i) #|2⁄𝜋 × arctan ((𝜋 2) × 𝑉𝑡+1
    if np.random.uniform() < velocityProbability:
      cPt_i = 1
    else:
      cPt_i = 0

    wPt_i = p.wPosition + wVt_i

    print("wPt_i:", wPt_i)
    # -- for END --

## Data Processing

In [29]:
CSV_NAME = "parkinsons.csv"
CSV_PATH = "E:/Perfil/OneDrive/Escritorio/MrRobot/IITV/4/TFG/data-Vito-PC/" + CSV_NAME
df = pd.read_csv(CSV_PATH, sep=" ", header=None)

end = df.shape[1]
X = df.iloc[:, :end-1] # iloc[rows,[cols_start,cols_end)] <- Dataframe object
Y = df.iloc[:, end-1] # iloc[rows,col_index] <- Dataframe series

# IMPORTANT, otherwise we would be using DataFrames and matrix operations won't work
X = X.to_numpy()
Y = Y.to_numpy()
J = len(np.unique(Y))

N,K = X.shape[0],X.shape[1] 

In [30]:
# Scaling (min-max normalization)
X_scaled = (X - X.min()) / (X.max() - X.min())

# First partition training/test
X_train, X_test, y_train, y_test = train_test_split(X_scaled,Y, test_size=0.33)

# Second partition validation
X_trainVal, X_testVal, y_trainVal, y_testVal = train_test_split(X_train,y_train, test_size=0.33)

## ELM

In [31]:
# Outputs the fitness for a given Gene
def ComputeChromosomeFitness(weigths, D,C,trainingX, trainingY, testX, testY):
    # W
    W = weigths 

    # Bias
    Bias = np.random.uniform(low= 0, high= 1, size=[D,1])

    # Amplify the matrix to the size of Xtraining and Xtest
    BiasTrainingMatrix = np.matlib.repmat(Bias,1,trainingX.shape[0])
    BiasTestMatrix = np.matlib.repmat(Bias,1,testX.shape[0])

    # Transpose BiasMatrix
    BiasTrainingMatrix = np.transpose(BiasTrainingMatrix)
    BiasTestMatrix = np.transpose(BiasTestMatrix)

    # H (Sigmoide function) 
    activationTraining = multiplyMatrixes(trainingX, W) + BiasTrainingMatrix
    activationTest = multiplyMatrixes(testX, W) + BiasTestMatrix
    
    H_Training = dotDivide( 1, (1 + np.exp(-activationTraining)) )    
    H_Test = dotDivide( 1, (1 + np.exp(-activationTest)) )

    # Beta
    aux = multiplyMatrixes( np.transpose(H_Training) , H_Training )
    delta = np.identity(aux.shape[0]) * 10e-3

    inverse = np.linalg.inv(dotDivide(np.identity(D) , C) + delta + aux)

    # Complete Formula: inv( (eye(D) ./ C) + delta + aux) * H_Training' * Ytraining;
    Beta = multiplyMatrixes(inverse, np.transpose(H_Training))
    Beta = multiplyMatrixes(Beta, trainingY)

    # Output
    Y_predicted = multiplyMatrixes(H_Test, Beta)
    fitness = np.linalg.norm(Y_predicted - testY)

    return fitness

In [32]:
# Updates the fitnesses of a given population of Genes and also, set the new best Gene 
def ComputePopulationFitness(population, D,C,trainingX, trainingY, testX, testY):
  # Clear previous fitnesses
  population.fitnessList.clear()

  for index,individual in enumerate(population.geneList):
    f_i = ComputeChromosomeFitness(individual.w, D, C, trainingX, trainingY, testX, testY)
     
      # Set fitness
    individual.setFitness(f_i)
    #print("Best Gene Fi middle:", population.bestGene.fitness)
    population.addFitnessToList(f_i)

      # Update Best Gene in Population
    if(individual.fitness < population.bestGene.fitness):
      population.bestGene = individual

In [33]:
# Updates the fitnesses of a given population of Genes and also, set the new best Gene 
def ComputeSwarmFitness(swarm, D,C,trainingX, trainingY, testX, testY):
  # Clear previous fitnesses
  swarm.fitnessList.clear()

  for index,particle in enumerate(swarm.geneList):
  # - - - - - - - - - - - - - - - - - -
    f_i = ComputeChromosomeFitness(particle.wPosition, D, C, trainingX, trainingY, testX, testY)
      
      # Set fitness
    particle.setFitness(f_i)
    swarm.addFitnessToList(f_i)

      # Update Pb_i for each particle and find the best gene 
    if particle.fitness < particle.bestFitness:
      particle.bestFitness = particle.fitness
      
      if particle.fitness < swarm.bestGene.fitness:
        swarm.bestGene = particle
  # - - - - - - - - - - - - - - - - - -
    # Update Gb of the swarm
  swarm.setBestSwarmChromosome(swarm.bestGene.c)
  swarm.setBestSwarmWeigths(swarm.bestGene.w)
  print("Global best fitness: ", swarm.bestGene.fitness)

In [34]:
def TrainModelAndOutputResults(weigths, D,C,trainingX, trainingY, testX, testY):
      # W
    W = weigths 

    # Bias
    Bias = np.random.uniform(low= 0, high= 1, size=[D,1])

    # Amplify the matrix to the size of Xtraining and Xtest
    BiasTrainingMatrix = np.matlib.repmat(Bias,1,trainingX.shape[0])
    BiasTestMatrix = np.matlib.repmat(Bias,1,testX.shape[0])

    # Transpose BiasMatrix
    BiasTrainingMatrix = np.transpose(BiasTrainingMatrix)
    BiasTestMatrix = np.transpose(BiasTestMatrix)

    # H (Sigmoide function) 
    activationTraining = multiplyMatrixes(trainingX, W) + BiasTrainingMatrix
    activationTest = multiplyMatrixes(testX, W) + BiasTestMatrix
    
    H_Training = dotDivide( 1, (1 + np.exp(-activationTraining)) )    
    H_Test = dotDivide( 1, (1 + np.exp(-activationTest)) )

    # Beta
    aux = multiplyMatrixes( np.transpose(H_Training) , H_Training )
    delta = np.identity(aux.shape[0]) * 10e-3

    inverse = np.linalg.inv(dotDivide(np.identity(D) , C) + delta + aux)

    # Complete Formula: inv( (eye(D) ./ C) + delta + aux) * H_Training' * Ytraining;
    Beta = multiplyMatrixes(inverse, np.transpose(H_Training))
    Beta = multiplyMatrixes(Beta, trainingY)

    # Prediction
    predictedLabels = multiplyMatrixes(H_Test, Beta)

    correctPrediction = 0
    for i in range(testY.shape[0]):
      if np.round(predictedLabels[i]) == testY[i]:
        correctPrediction +=1
    
    CCR = correctPrediction / testY.shape[0]
    return CCR * 100

## Genetic Algorithm

In [35]:
def GeneticAlgorithm(gaPopulation, nGenerations, optimalD, optimalC, trainX, trainY, testX, testY):
  t = 0
  while t < nGenerations:
      # Reproduction
    ReproducePopulation(gaPopulation)  
      # Compute Fitness
    ComputePopulationFitness(gaPopulation, optimalD, optimalC, trainX, trainY, testX, testY)
      # Selector operator
    gaPopulation = RouletteWheelSelection(gaPopulation)
      # Continue iterating
    t += 1
  
  return gaPopulation

## Particle Swarm Optimization

In [36]:
def PSOAlgorithm(swarm,maxIterations, w, c1, c2, dampingW, optimalD, optimalC, trainX, trainY, testX, testY):
  t = 0
  while t < maxIterations:
      # Evaluate Particles' Fitness and update Pb_i and Gb_i
    ComputeSwarmFitness(swarm, optimalD, optimalC, trainX, trainY, testX, testY)

      # Update Velocity and Position of each particle
    UpdateVelocitiesAndPositionsPSO(swarm, w, c1, c2)

      # Damping inertia
    w = w * dampingW

      # Increase step
    t += 1

  return 1

## Coral Reef Optimization

In [37]:
def CROAlgorithm():
  t = 0
  return 1
  '''
  while t < coralGenerations:
    # Update variables
    F_broadcast = np.random.uniform() # broadcast fraction
    F_brooding = 1 - F_broadcast # brooding fraction
    Pd = 0.2 # predation probability
    
    # Broadcast and Brooding

    # Asexual reproduction

    # Predation 

    # Fitness Evaluation

    # Increase step
    t += 1
    # - end WHILE

  print("Best CRO: ", "coral X")
  '''